In [ ]:

from keras.datasets import mnist
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dense
from keras.layers import Flatten
from keras.optimizers import SGD
import pandas as pd

# load train and test dataset
# load and prepare the image
def load_image(filename):
    # load the image
    img = load_img(filename, grayscale=True, target_size=(400, 400))
    # convert to array
    img = img_to_array(img)
    # reshape into a single sample with 1 channel
    img = img.reshape(1, 400, 400, 1)
    # prepare pixel data
    img = img.astype('float32')
    img = img / 255.0
    return img

def load_dataset(df):
    # load dataset
    # reshape dataset to have a single channel
    trainX = df['train_imgs'].reshape((df['train_imgs'].shape[0], 400, 400, 1))
    testX = df['test_imgs'].reshape((df['test_imgs'].shape[0], 400, 400, 1))
    # one hot encode target values
    trainY = to_categorical(df['train_lbls'])
    testY = to_categorical(df['test_lbls'])
    return trainX, trainY, testX, testY

# scale pixels
def prep_pixels(train, test):
    # convert from integers to floats
    train_norm = train.astype('float32')
    test_norm = test.astype('float32')
    # normalize to range 0-1
    train_norm = train_norm / 255.0
    test_norm = test_norm / 255.0
    # return normalized images
    return train_norm, test_norm

# define cnn model
def define_model():
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=(400, 400, 1)))
    model.add(MaxPooling2D((2, 2))) # important feature extraction, autoencoder
    # https://keras.io/api/layers/pooling_layers/max_pooling2d/
    model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform')) # convolutional layer
    model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform')) 
    model.add(MaxPooling2D((2, 2))) # 
    model.add(Flatten()) # convert into a one vector
    model.add(Dense(100, activation='relu', kernel_initializer='he_uniform')) # output as 100 values, to interpret the features
    model.add(Dense(10, activation='softmax')) # output as 10 classes
    # compile model
    opt = SGD(lr=0.01, momentum=0.9) # adding optimizer, stochastic Gradient descent optimizer
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    return model
# evaluate the model 
# evaluate a model using k-fold cross-validation
def evaluate_model(dataX, dataY, n_folds=5):
    scores, histories = list(), list()
    # prepare cross validation
    kfold = KFold(n_folds, shuffle=True, random_state=1)
    # enumerate splits
    for train_ix, test_ix in kfold.split(dataX):
        # define model
        model = define_model()
        # select rows for train and test
        trainX, trainY, testX, testY = dataX[train_ix], dataY[train_ix], dataX[test_ix], dataY[test_ix]
        # fit model
        history = model.fit(trainX, trainY, epochs=10, batch_size=32, validation_data=(testX, testY), verbose=0)
        # evaluate model
        _, acc = model.evaluate(testX, testY, verbose=0)
        print('> %.3f' % (acc * 100.0))
        # stores scores
        scores.append(acc)
        histories.append(history)
    return scores, histories

# summarize model performance
def summarize_performance(scores):
    # print summary
    print('Accuracy: mean=%.3f std=%.3f, n=%d' % (mean(scores)*100, std(scores)*100, len(scores)))
    # box and whisker plots of results
    pyplot.boxplot(scores)
    pyplot.show()
    
    
def saveModel():
    
    model.fit(trainX, trainY, epochs=10, batch_size=32, verbose=0)
    # save model
    model.save('final_model.h5')

# entry point, run the test harness
saveModel()



In [ ]:
# make a prediction for a new image.
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.models import load_model

# load and prepare the image
def load_image(filename):
    # load the image
    img = load_img(filename, grayscale=True, target_size=(400, 400))
    # convert to array
    img = img_to_array(img)
    # reshape into a single sample with 1 channel
    img = img.reshape(1, 400, 400, 1)
    # prepare pixel data
    img = img.astype('float32')
    img = img / 255.0
    return img

# load an image and predict the class
def run():
    # load the image
    img = load_image('face1.png')
    # load model
    model = load_model('final_model.h5')
    # predict the class
    attentiveness = model.predict_classes(img)
    print(attentiveness[0])

# entry point, run the example
run()